In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

from datasets import *
from models import *
from util import *
from training import *

## Load data

In [2]:
p_pheno = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/pheno_01-12-21.csv'
p_conn = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/connectomes_01-12-21.csv'

data1 = caseControlDataset('SZ',p_pheno,p_conn)
data2 = caseControlDataset('BIP',p_pheno,p_conn)

task_to_name = {'task1':'SZ','task2':'BIP'}

/home/harveyaa/miniconda3/envs/MTL/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (7,8,12,13,14,19,20,24,27,31,42,43,44,49,51,59,60,62,64,65,68,101,121,163) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.
/home/harveyaa/miniconda3/envs/MTL/lib/python3.7/site-packages/ipykernel_launcher.py:5: DtypeWarning: Columns (7,8,12,13,14,19,20,24,27,31,42,43,44,49,51,59,60,62,64,65,68,101,121,163) have mixed types.Specify dtype option on import or set low_memory=False.
  """


In [3]:
train_data1, test_data1 = split_data(data1)
train_data2, test_data2 = split_data(data2)

trainloader1 = DataLoader(train_data1, batch_size=16, shuffle=True)
testloader1 = DataLoader(test_data1, batch_size=16, shuffle=True)

trainloader2 = DataLoader(train_data2, batch_size=16, shuffle=True)
testloader2 = DataLoader(test_data2, batch_size=16, shuffle=True)

loss_fns = {task_to_name['task1']:nn.CrossEntropyLoss(),task_to_name['task2']:nn.CrossEntropyLoss()}

In [4]:
model = HPSModel(encoder().double(),
                {task_to_name['task1']:head().double(),task_to_name['task2']:head().double()},
                loss_fns)

In [5]:
trainloaders = {task_to_name['task1']:trainloader1, task_to_name['task2']:trainloader2}
testloaders = {task_to_name['task1']:testloader1, task_to_name['task2']:testloader2}

In [6]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

device = 'cpu'

In [9]:
trainer = Trainer(optimizer,10)

trainer.fit(model,trainloaders)

In [10]:
testMTL(testloaders, model, loss_fns)

SZ
Test Error: 
 Accuracy: 57.0%, Avg loss: 0.042024 

BIP
Test Error: 
 Accuracy: 59.4%, Avg loss: 0.042786 

